In [ ]:
proj_id = 'cmat-315920'
root_path = '/home/jupyter'
code_path = root_path + '/MathVGerrmandering_CMAT_2021/src/'
for f in ['Core', 'Gerry', 'Variable', 'Crosswalks', 'Assignments', 'Shapes',
          'Census', 'Elections', 'Votes', 'Combined', 'Edges', 'Nodes', 'Graph', 'Districts']:
    %run -i {code_path+f+'.py'}

refresh_all = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

refresh_tbl = (
#     'crosswalks',
#     'assignments',
#     'shapes',
#     'census',
#     'elections',
#     'votes_all',
#     'votes_hl',
#     'combined',
#     'edges',
#     'nodes',
#     'graph',
)

self = Gerry(abbr = 'TX',
             level = 'cntyvtd',
             district_type='cd',
             simplification=100,
             num_colors=10,
             election_filters=(
                 "office='President' and race='general'",
                 "office='USSen' and race='general'",
#                  "office like 'USRep%' and race='general'",
             ),
             refresh_tbl=refresh_tbl, refresh_all=refresh_all,
             num_steps=50,
            )

start = time.time()

# idx = self.nodes.df.nlargest(2, 'total_pop').index
# self.nodes.df.loc[idx[0], 'cd'] = '37'
# self.nodes.df.loc[idx[1], 'cd'] = '38'

self.MCMC()
self.agg_plans(agg_polygon_steps=True)
#                (0,10))#True)#agg_polygon_steps=list(range(3,15)))
self.stack_plans()

elapsed = time.time() - start
h, m = divmod(elapsed, 3600)
m, s = divmod(m, 60)
print(f'{int(h)}hrs {int(m)}min {s:.2f}sec elapsed')

In [ ]:
A = read_table(self.assignments.tbl)
A['ct'] = A.groupby('cnty')['cd'].transform('nunique')
mask = A['ct'] == 1
A['cntyvtd2'] = A['cntyvtd'].copy()
A.loc[mask, 'cntyvtd2'] = A.loc[mask, 'cnty']
print(A.groupby('cntyvtd2').size(), A.groupby('cntyvtd').size(), self.graph.graph.size())